In [3]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
import numpy as np
lista=['XRPUSDT',	
'XTZUSDT',	
'OMGUSDT',	
'CRVUSDT',	
'1INCHUSDT',
'MANAUSDT',
'FLOWUSDT',
'MAGICUSDT',
'MINAUSDT',
'PHBUSDT',	
'PERPUSDT',
'IDUSDT',	
'JOEUSDT',	
'HFTUSDT',	
'EDUUSDT',	
'SUIUSDT',	
'COMBOUSDT',
'MAVUSDT'	]

#lista=md.lista_de_monedas ()
lista=['SUIUSDT']

for symbol in lista:    
    try:
        #data = md.sigo_variacion_bitcoin(symbol)
        #data = md.estrategia_bb(symbol)
        data = md.estrategia_santa(symbol,tp_flag=True)
        resultado = md.backtesting(data, plot_flag = True)
        #if (
        #    (resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor']))
        #    and (resultado['Return [%]']/resultado['# Trades']) >=0.33 
        #    ):
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        pass

In [7]:
data = md.estrategia_santa('SUIUSDT',tp_flag=True)
resultado = md.backtesting(data, plot_flag = True)
data

RuntimeError: Indicator "get_bolli…" errored with exception: Column 'copy' not in data

In [5]:
salida=md.get_bollinger_bands_lower_upper(data)
salida

timestamp
2023-07-12 13:30:00         NaN
2023-07-12 13:45:00         NaN
2023-07-12 14:00:00         NaN
2023-07-12 14:15:00         NaN
2023-07-12 14:30:00         NaN
                         ...   
2023-07-22 22:15:00    0.161763
2023-07-22 22:30:00    0.161132
2023-07-22 22:45:00    0.160276
2023-07-22 23:00:00    0.160200
2023-07-22 23:15:00    0.160728
Name: upper, Length: 1000, dtype: float64

In [1]:
import modulos as md
symbol='BTCUSDT'
data = md.obtiene_historial(symbol,'15m')
data= md.get_bollinger_bands(data)
data

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Open Time,Open,High,Low,Close,Volume,ema20,ema50,ema200,atr,n_atr,upper,lower
timestamp,,,,,,,,,,,,,
2023-07-12 13:30:00,2023-07-12 13:30:00,30658.01,30735.08,30461.59,30587.80,1908.84234,NaN,NaN,NaN,NaN,50,NaN,NaN
2023-07-12 13:45:00,2023-07-12 13:45:00,30587.79,30683.96,30541.99,30666.33,759.11550,NaN,NaN,NaN,NaN,50,NaN,NaN
2023-07-12 14:00:00,2023-07-12 14:00:00,30666.33,30850.33,30652.00,30799.69,1129.27820,NaN,NaN,NaN,NaN,50,NaN,NaN
2023-07-12 14:15:00,2023-07-12 14:15:00,30799.69,30814.99,30725.93,30732.00,432.18615,NaN,NaN,NaN,NaN,50,NaN,NaN
2023-07-12 14:30:00,2023-07-12 14:30:00,30731.99,30811.23,30731.99,30791.99,377.83133,NaN,NaN,NaN,NaN,50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-22 22:15:00,2023-07-22 22:15:00,29824.31,29824.32,29800.00,29814.52,97.81251,29836.071508,29856.149538,29889.195844,18.302504,50,29848.986016,29811.020984
2023-07-22 22:30:00,2023-07-22 22:30:00,29814.51,29818.08,29801.25,29818.07,124.18592,29834.357079,29854.656223,29888.488124,18.197325,50,29847.339257,29810.128743
2023-07-22 22:45:00,2023-07-22 22:45:00,29818.08,29818.08,29785.72,29808.00,130.70540,29831.846881,29852.826567,29887.687247,19.208944,50,29847.482187,29807.050813
